In [2]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

# load data
train = fetch_20newsgroups(subset='train', shuffle=False)
test = fetch_20newsgroups(subset='test', shuffle=False)

# parse data to pandas DataFrames
newsgroup_test = pd.DataFrame({'article':test.data, 'class_index':test.target})
newsgroup_train = pd.DataFrame({'article':train.data, 'class_index':train.target})



In [5]:
# load labels with keywords
labels = pd.read_csv('20newsgroups_keywords.csv',sep=';')
print(labels)

   class_index          class_name            keywords
0            8     rec.motorcycles    Bikes Motorcycle
1            9  rec.sport.baseball            Baseball
2           10    rec.sport.hockey              Hockey
3           11           sci.crypt  Encryption Privacy


In [6]:
# split keywords by separator and save them as array
labels['keywords'] = labels['keywords'].apply(lambda x: x.split(' '))

# convert description keywords to lowercase
labels['keywords'] = labels['keywords'].apply(lambda description_keywords: [keyword.lower() for keyword in description_keywords])

# get number of keywords for each class
labels['number_of_keywords'] = labels['keywords'].apply(lambda row: len(row))

# lets check our keywords
print(labels)

   class_index          class_name               keywords  number_of_keywords
0            8     rec.motorcycles    [bikes, motorcycle]                   2
1            9  rec.sport.baseball             [baseball]                   1
2           10    rec.sport.hockey               [hockey]                   1
3           11           sci.crypt  [encryption, privacy]                   2


In [7]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags
from gensim.models.doc2vec import TaggedDocument

# doc: document text string
# returns tokenized document
# strip_tags removes meta tags from the text
# simple preprocess converts a document into a list of lowercase tokens, ignoring tokens that are too short or too long
# simple preprocess also removes numerical values as well as punktuation characters
def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)

# add data set type column
newsgroup_train['data_set_type'] = 'train'
newsgroup_test['data_set_type'] = 'test'

# concat train and test data
newsgroup_full_corpus = pd.concat([newsgroup_train,newsgroup_test]).reset_index(drop=True)

# reduce dataset to only articles that belong to classes where we defined our keywords
newsgroup_full_corpus = newsgroup_full_corpus[newsgroup_full_corpus['class_index'].isin(list(labels['class_index']))]

# tokenize and tag documents for Lbl2Vec training
newsgroup_full_corpus['tagged_docs'] = newsgroup_full_corpus.apply(lambda row: TaggedDocument(tokenize(row['article']), [str(row.name)]), axis=1)

# add doc_key column
newsgroup_full_corpus['doc_key'] = newsgroup_full_corpus.index.astype(str)

# add class_name column
newsgroup_full_corpus = newsgroup_full_corpus.merge(labels, left_on='class_index', right_on='class_index', how='left')

print(newsgroup_full_corpus.head())

                                             article  class_index  \
0  From: cubbie@garnet.berkeley.edu (            ...            9   
1  From: crypt-comments@math.ncsu.edu\nSubject: C...           11   
2  From: george@minster.york.ac.uk\nSubject: Non-...           11   
3  From: williac@govonca.gov.on.ca (Chris William...           10   
4  From: ayari@judikael.loria.fr (Ayari Iskander)...           10   

  data_set_type                                        tagged_docs doc_key  \
0         train  ([from, cubbie, garnet, berkeley, edu, subject...       0   
1         train  ([from, crypt, comments, math, ncsu, edu, subj...       2   
2         train  ([from, george, minster, york, ac, uk, subject...      11   
3         train  ([from, williac, govonca, gov, on, ca, chris, ...      12   
4         train  ([from, ayari, judikael, loria, fr, ayari, isk...      15   

           class_name               keywords  number_of_keywords  
0  rec.sport.baseball             [baseball]     

In [9]:
!pip install lbl2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 10.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyli

In [10]:
from lbl2vec import Lbl2Vec

# init model with parameters
Lbl2Vec_model = Lbl2Vec(keywords_list=list(labels.keywords), tagged_documents=newsgroup_full_corpus['tagged_docs'][newsgroup_full_corpus['data_set_type'] == 'train'], label_names=list(labels.class_name), similarity_threshold=0.43, min_num_docs=100, epochs=10)

# train model
Lbl2Vec_model.fit()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2024-04-11 10:24:35,786 - Lbl2Vec - INFO - Train document and word embeddings
INFO:Lbl2Vec:Train document and word embeddings
2024-04-11 10:25:02,975 - Lbl2Vec - INFO - Train label embeddings
INFO:Lbl2Vec:Train label embeddings


In [11]:
from sklearn.metrics import f1_score

# predict similarity scores
model_docs_lbl_similarities = Lbl2Vec_model.predict_model_docs()

# merge DataFrames to compare the predicted and true category labels
evaluation_train = model_docs_lbl_similarities.merge(newsgroup_full_corpus[newsgroup_full_corpus['data_set_type'] == 'train'], left_on='doc_key', right_on='doc_key')
y_true_train = evaluation_train['class_name']
y_pred_train = evaluation_train['most_similar_label']

print('F1 score:',f1_score(y_true_train, y_pred_train, average='micro'))

2024-04-11 10:25:54,370 - Lbl2Vec - INFO - Get document embeddings from model
INFO:Lbl2Vec:Get document embeddings from model
2024-04-11 10:25:54,419 - Lbl2Vec - INFO - Calculate document<->label similarities
INFO:Lbl2Vec:Calculate document<->label similarities


F1 score: 0.895397489539749


In [12]:
# predict similarity scores of new test documents (they were not used during Lbl2Vec training)
new_docs_lbl_similarities = Lbl2Vec_model.predict_new_docs(tagged_docs=newsgroup_full_corpus['tagged_docs'][newsgroup_full_corpus['data_set_type']=='test'])

# merge DataFrames to compare the predicted and true topic labels
evaluation_test = new_docs_lbl_similarities.merge(newsgroup_full_corpus[newsgroup_full_corpus['data_set_type']=='test'], left_on='doc_key', right_on='doc_key')
y_true_test = evaluation_test['class_name']
y_pred_test = evaluation_test['most_similar_label']

print('F1 score:',f1_score(y_true_test, y_pred_test, average='micro'))

2024-04-11 10:26:20,281 - Lbl2Vec - INFO - Calculate document embeddings
INFO:Lbl2Vec:Calculate document embeddings
2024-04-11 10:26:22,712 - Lbl2Vec - INFO - Calculate document<->label similarities
INFO:Lbl2Vec:Calculate document<->label similarities


F1 score: 0.8773584905660378
